In [1]:
from llm.llm import AWSBedrockLLm

sonnet35 = "anthropic.claude-3-5-sonnet-20240620-v1:0"

llm = AWSBedrockLLm(model_id=sonnet35)

print(llm.model)

client=<botocore.client.BedrockRuntime object at 0x7f58d8fdbb00> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')


In [2]:
llm.invoke("HI")

AIMessage(content='Hello! How can I assist you today? Feel free to ask any questions or let me know if you need help with anything.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'd726b096-a7e0-499b-b418-d2f6a5069fa0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 24 Dec 2025 03:55:47 GMT', 'content-type': 'application/json', 'content-length': '314', 'connection': 'keep-alive', 'x-amzn-requestid': 'd726b096-a7e0-499b-b418-d2f6a5069fa0'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [1031]}, 'model_name': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='lc_run--019b4e7f-a41b-7301-add2-d387c3154c09-0', usage_metadata={'input_tokens': 9, 'output_tokens': 29, 'total_tokens': 38, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

In [3]:
from documents.schema import all_docs


In [4]:
from vectorstore.vectorstore import VectorStore


In [5]:
vecstore = VectorStore(docs=all_docs)

In [6]:
from prompt.prompt import *



Summarize the following concisely focusing only on main points:HII


In [7]:
template = """ You are a SQL expert, use the following retrieved docs to generate sql to answer the user question. Only reply with a executaible postgres sql
Docs: 
{docs}

User Query: 
{query}
"""
ragprompt = CustomPrompt(template=template)

In [8]:
userquery = "Count number of clients per state"

In [9]:
retrieved_docs  = vecstore.retrieve(userquery)

In [10]:
retrieved_docs

[Document(id='b74d74ac-ef50-42a9-8a06-95c90c278706', metadata={'type': 'metric', 'name': 'Client Census'}, page_content='  \nName: Client Census\nDescription: count of all the client for a given week from sunday to saturday\nSQL: SELECT COUNT(DISTINCT client_key) FROM dw_prod_report.vw_hours_and_client_census_weekly WHERE period_begin_date = (SELECT CURRENT_DATE - (EXTRACT(DOW FROM CURRENT_DATE)::integer + 7) AS last_full_week_sunday) AND CLIENT_SERVED_FLAG = TRUE AND client_key IS NOT NULL\n'),
 Document(id='22acfd8c-6dd7-435b-bee2-40c9db40904e', metadata={'type': 'metric_comparison', 'name': 'Client Census Week-over-Week Change', 'optimized': True, 'topic': 'census', 'granularity': 'weekly', 'dimension': 'state'}, page_content='\nName: Client Census Week-over-Week Comparison\nDescription: Count of distinct clients served per state for the last two full weeks (Sunday to Saturday), and the change in client count between those two weeks.\nSQL: WITH census_counts AS (\n  SELECT \n    sta

In [11]:
from util import *

In [12]:
formatted_docs = format_docs(retrieved_docs)
formatted_docs

"  \nName: Client Census\nDescription: count of all the client for a given week from sunday to saturday\nSQL: SELECT COUNT(DISTINCT client_key) FROM dw_prod_report.vw_hours_and_client_census_weekly WHERE period_begin_date = (SELECT CURRENT_DATE - (EXTRACT(DOW FROM CURRENT_DATE)::integer + 7) AS last_full_week_sunday) AND CLIENT_SERVED_FLAG = TRUE AND client_key IS NOT NULL\n\n\n\nName: Client Census Week-over-Week Comparison\nDescription: Count of distinct clients served per state for the last two full weeks (Sunday to Saturday), and the change in client count between those two weeks.\nSQL: WITH census_counts AS (\n  SELECT \n    state,\n    period_begin_date,\n    COUNT(DISTINCT client_key) AS client_count\n  FROM dw_prod_report.vw_hours_and_client_census_weekly_partitioned\n  WHERE CLIENT_SERVED_FLAG = TRUE\n    AND client_key IS NOT NULL\n    AND state IS NOT NULL\n    AND period_begin_date IN (\n      CURRENT_DATE - (EXTRACT(DOW FROM CURRENT_DATE)::integer + 7),\n      CURRENT_DATE

In [13]:
from util import SQLQuery

In [14]:
llm_structure = AWSBedrockLLm(model_id=sonnet35, structured_output=SQLQuery)

In [15]:
ragresponse_structure = llm_structure.invoke(ragprompt.get({'docs': formatted_docs, 'query': userquery}))


In [16]:
ragresponse = llm.invoke(ragprompt.get({'docs': formatted_docs, 'query': userquery}))


In [17]:
ragresponse_structure

SQLQuery(question='Count number of clients per state', sql='SELECT state, COUNT(DISTINCT client_key) AS client_count\nFROM dw_prod_report.vw_hours_and_client_census_monthly\nWHERE CLIENT_SERVED_FLAG = TRUE\n  AND client_key IS NOT NULL\n  AND state IS NOT NULL\nGROUP BY state\nORDER BY client_count DESC')

In [18]:
ragresponse.content

"Here's the SQL query to count the number of clients per state:\n\n```sql\nSELECT \n    state, \n    COUNT(DISTINCT client_key) AS client_count\nFROM \n    dw_prod_report.vw_hours_and_client_census_monthly\nWHERE \n    CLIENT_SERVED_FLAG = TRUE \n    AND client_key IS NOT NULL \n    AND state IS NOT NULL\nGROUP BY \n    state\nORDER BY \n    client_count DESC;\n```\n\nThis query:\n1. Selects from the monthly view to get the most comprehensive data.\n2. Counts distinct client keys to avoid duplicates.\n3. Filters for active clients (CLIENT_SERVED_FLAG = TRUE) and ensures both client_key and state are not null.\n4. Groups the results by state.\n5. Orders the results by the client count in descending order to show states with the most clients first."

In [23]:
ragresponse_structure.sql

'SELECT state, COUNT(DISTINCT client_key) AS client_count\nFROM dw_prod_report.vw_hours_and_client_census_monthly\nWHERE CLIENT_SERVED_FLAG = TRUE\n  AND client_key IS NOT NULL\n  AND state IS NOT NULL\nGROUP BY state\nORDER BY client_count DESC'